In [2]:
import numpy as np
import pandas as pd
from scipy import integrate
import matplotlib.pyplot as plt

V1 is the forward barrier energy

V2 is the reverse barrier energy

nu is the linear frequency of the reaction mode 

In [3]:
h = 6.6261e-34
T=298
kB = 1.3806e-23
beta = 1/(kB*T)

Using TS IRC Pass 2 data:

V1 = 0.014181 Hartree = 6.182547294e-20 J

V2 = 0.0272012 Hartree = 1.185901597e-19 J

nu = 4.838370768e13 Hz

## Johnston and Heicklen 1961

In [4]:
def alpha_1(E, V1, V2, nu):
    return((2*np.pi*V1)/(h*nu))

In [5]:
def alpha_2(E, V1, V2, nu):
    return((2*np.pi*V2)/(h*nu))

In [231]:
# def xi(E, V1, V2, nu):
#     return(E/V1)

In [232]:
# def Kappa(E, V1, V2, nu):
#     al_2 = alpha_1(E, V1, V2, nu)
#     al_1 = alpha_2(E, V1, V2, nu)
#     x = xi(E, V1, V2, nu)
    
#     a = 2*np.sqrt(al_1*x)*((np.sqrt(1/al_1) + np.sqrt(1/al_2))**(-1))
#     b = 2*np.sqrt((1+x)*al_1 - al_2)*((np.sqrt(1/al_1)+np.sqrt(1/al_2))**(-1))
#     d = 2*np.sqrt(al_1*al_2-((np.pi)**2)/4)
    
#     result = 1 - ((np.cosh(a-b) + np.cosh(d))/(np.cosh(a+b) + np.cosh(d)))
#     return(result)

In [233]:
# Kappa(E=-1e-19, V1=6.18e-21, V2=1.186e-20, nu=4.838e13)

In [234]:
# x = np.linspace(0, 1e-16, 100000)
# value = np.zeros(len(x))
# for i in range(len(x)):
#     value[i] = integrand(E=x[i], V1=6.18e-20, V2=1.186e-19, nu=4.838e13)
# plt.plot(x, value)
# value[-500]

In [235]:
# def integrand(E, V1, V2, nu):
#     return(np.exp(-E*beta)*Kappa(E, V1, V2, nu))

In [236]:
# def quantum_correction(V1, V2, nu):
#     trial = V1/1000
#     converged = False
#     while converged == False:
#         if integrand(trial, V1, V2, nu) > 1e-20:
#             trial += V1/100
#         else:
#             converged = True
#     print(trial)
#     return(beta*np.exp(V1*beta)*integrate.fixed_quad(integrand, -1e-19, 1e-15, args=(V1, V2, nu), n=6)[0])

In [237]:
# quantum_correction(V1=6.18e-21, V2=1.186e-20, nu=4.838e13)

## Brown 1981

In [252]:
al_1 = 12.1
al_2 = 23.2

In [238]:
def epsilon(E, V1, V2, nu):
    return((E-V1)/(kB*T))

In [253]:
def Kappa(eps, V1, V2, nu):
#     al_1 = alpha_1(E, V1, V2, nu)
#     al_2 = alpha_2(E, V1, V2, nu)
    u_star = (h*nu)/(kB*T)
    c = (np.pi/8)*u_star*((np.sqrt(1/al_1) + np.sqrt(1/al_2))**2)
    
    v1 = V1/(kB*T)
    v2 = V2/(kB*T)
    a1 = np.pi*np.sqrt((eps + v1)/c)
    a2 = np.pi*np.sqrt((eps + v2)/c)
    
    d = np.sqrt(4*np.pi*al_1*al_2 - (np.pi)**2)
    D = np.cosh(d)
    
    result = (np.cosh(a1 + a2) - np.cosh(a1 - a2))/(np.cosh(a1 + a2) + D)
    return(result)

In [288]:
Kappa(eps=1e3, V1=6.18e-20, V2=1.186e-19, nu=4.838e13)*np.exp(15)

3269017.3724721107

In [286]:
def integrand(eps, V1, V2, nu):
    return(np.exp(-eps)*Kappa(eps, V1, V2, nu))

In [276]:
def quantum_correction(V1, V2, nu):
    epsilon_0 = -V1/(kB*T)
    print(integrate.quad(integrand, epsilon_0, 100, args=(V1, V2, nu)))

In [277]:
quantum_correction(V1=6.18e-20, V2=1.186e-19, nu=4.838e13)

(1.51312918237432e-10, 5.298975295561361e-11)


## Brown 1981 (transcribed from FORTRAN routine)

In [11]:
def eckart(alpha1, alpha2, u): #u is imaginary barrier frequency * hc/k_BT
    Data_X = [-.9324695,-.6612094,-.2386192,.2386192,.6612094,.9324695]
    Data_W = [.1713245,.3607616,.4679139,.4679139,.3607616,.1713245]
    
    c = (np.pi*u/8)*(((alpha1**-0.5)+(alpha2**-0.5))**2)
    
    v1 = (u/(2*np.pi))*alpha1
    v2 = (u/(2*np.pi))*alpha2
    print(v1)
    
    d = 4*alpha1*alpha2-(np.pi)**2
    if d < 0:
        D = np.cos((-d)**0.5)
    else:
        D = np.cosh((d)**0.5)
    
    if v2 >= v1:
        EZ = -v1
    else:
        EZ = -v2
        
    EB = min((c*((np.log(2*(1+D)/0.014)/(2*np.pi))**2)-0.5*(v1 + v2)), 3.2) 
    
    EM = 0.5*(EB - EZ)
    
    EP = 0.5*(EB + EZ)
    
    G = 0
    for i in range(len(Data_X)):
        E = EM*Data_X[i] + EP
        A1 = np.pi*(((E+v1)/c)**0.5)
        A2 = np.pi*(((E+v2)/c)**0.5)
        FP = np.cosh(A1+A2)
        FM = np.cosh(A1-A2)
        G += Data_W[i]*np.exp(-E)*(FP-FM)/(FP+D)
    G = EM*G+np.exp(-EB)
    
    return(G)

In [12]:
eckart(3.7065, 18.019, 5.9819)

3.5287694483027416


3.0805164180642417